# 0. Getting setup

In [ ]:
# !git clone https://github.com/timesler/facenet-pytorch.git facenet_pytorch

In [ ]:
# from .autonotebook import tqdm as notebook_tqdm
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training
import torch
from torch import nn
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import numpy as np
from torchinfo import summary
import random
from PIL import Image
import matplotlib.pyplot as plt

Determine if an nvidia GPU is available

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

# 2. Create Datasets and DataLoaders

In [ ]:
from pathlib import Path
# Setup path to data folder
image_path = Path("data/")

if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print("Please create data from add_person.py and addRandomFace.py")

In [ ]:
# Setup Dirs
train_dir = image_path / "train"
test_dir = image_path / "test"
valid_dir = image_path / "validation"

In [ ]:
# Create a transforms pipeline manually (required for torchvision < 0.13)
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    np.float32,
    # Flip the images randomly on the horizontal
    # transforms.RandomHorizontalFlip(p=0.2), # p = probability of flip, 0.2 = 20% chance
    # Randomly select a rectangle region in the input image and erase its pixels.
    # transforms.RandomErasing(p=0.2),
    # Randomly change the brightness, contrast, saturation and hue of an imag
    # transforms.ColorJitter(),
    # Turn the image into a torch.Tensor
     # this also converts all pixel values from 0 to 255 to be between 0.0 and 1.0
    fixed_image_standardization,
    transforms.ToTensor(),
])

In [ ]:
from modules import data_setup
# Create training and testing DataLoaders as well as get a list of class names
train_dataloader, valid_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                                 valid_dir=valid_dir,                  
                                                                                                 test_dir=test_dir,
                                                                                                 transform=manual_transforms, # resize, convert images to between 0 & 1 and normalize them
                                                                                                 batch_size=32) # set mini-batch size to 32

train_dataloader, valid_dataloader, test_dataloader, class_names

In [ ]:
examples = next(iter(train_dataloader))

In [ ]:
def plot_transformed_images(image_paths, transform, n=3, seed=42):
    """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths. 
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
    random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f) 
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            # Transform and plot image
            # Note: permute() will change shape of image to suit matplotlib 
            # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
            transformed_image = transform(f).permute(1, 2, 0) 
            ax[1].imshow(transformed_image) 
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

image_path_list = list(image_path.glob("*/*/*.jpg"))
plot_transformed_images(image_path_list, 
                        transform=manual_transforms, 
                        n=8)

In [ ]:
img, label = next(iter(train_dataloader))
# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

### Define Inception Resnet V1 module

In [ ]:
help(InceptionResnetV1)

In [ ]:
# Load pretrained resnet model
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(class_names)
).to(device)

In [ ]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=resnet, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

# Train model

In [ ]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [ ]:
epochs = 8
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, valid_dataloader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_dataloader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, valid_dataloader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()

In [ ]:
# Viewing TensorBoard in Jupyter and Google Colab Notebooks (uncomment to view full TensorBoard instance)
%load_ext tensorboard
%tensorboard --logdir runs

# Save model

In [ ]:
from modules.utils import save_model
# Create models directory (if it doesn't already exist), see: https://docs.python.org/3/library/pathlib.html#pathlib.Path.mkdir
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, # create parent directories if needed
                 exist_ok=True # if models directory already exists, don't error
)
# Create model save path
MODEL_NAME = "vggface.pth"

save_model(model=resnet,
            target_dir="models",
            model_name=MODEL_NAME)